In [ ]:
# Experimental Topology
# lockin AO 2 -> Analog Attenuator Vdd
# lockin AO 3 -> Analog Attenuator Vc (Control Voltages)
# Keithley Output -> Voltage divider with 1 MOhm, DAQmx AI1
# AWG Sync Marker Out -> DAQmx PFI0
# AWG Samp. Marker Out -> PSPL Trigger

## CSHE Phase Diagram with nTron pulses ##

In [1]:
cd ..

C:\Users\qlab\Documents\GitHub\pycontrol


In [2]:
from pycontrol.filters.io import WriteToHDF5, ProgressBar
import numpy as np
import itertools
import time
import importlib
import matplotlib.pyplot as plt
import analysis.switching as sw
from adapt import refine
from analysis.h5shell import h5shell

In [3]:
from experiments.Pulse_Switching_CSHE_nTron import nTronPhaseDiagramExperiment

In [4]:
exp = nTronPhaseDiagramExperiment()
exp.sample = "CSHE5-C1R3"
exp.comment = "Phase Diagram -  P to AP - Interations = 13 - Settle delay = 200us - Reset 0.2V 5ns"
exp.polarity = 1 # -1: AP to P; 1: P to AP
exp.iteration = 13
exp.field.value = -0.0074
exp.measure_current = 3e-6
exp.reset_amplitude = 0.2
exp.reset_duration  = 5.0e-9
exp.settle_delay    = 200e-6

In [5]:
wr = WriteToHDF5("data\CSHE-Switching\CSHE-Die5-C1R3\CSHE5-C1R3-nTron_P2AP_2016-07-19.h5")
edges = [(exp.daq_buffer, wr.data)]
exp.set_graph(edges)

In [6]:
coarse_ts = 1e-9*np.linspace(0.1, 10, 10) # List of durations
coarse_vs = np.linspace(0.5, 1.0, 10)

points    = [coarse_ts, coarse_vs]
points    = np.array(list(itertools.product(*points)))
exp.nTron_durations = points[:,0]
exp.nTron_voltages = points[:,1]
exp.init_instruments()
main_sweep = exp.add_unstructured_sweep([exp.nTron_duration, exp.nTron_voltage], points)

In [7]:
figs = []
t1 = time.time()
for i in range(exp.iteration):
    exp.init_progressbar(1,notebook=True)
    exp.reset()
    exp.run_sweeps()
    points, mean = sw.load_switching_data(wr.filename)
    figs.append(sw.phase_diagram_mesh(points, mean, title="Iteration={}".format(i)))
    new_points = refine.refine_scalar_field(points, mean, all_points=False,
                                criterion="integral", threshold = "one_sigma")
    if new_points is None:
        print("No more points can be added.")
        break
    #
    print("Elapsed time: {} min.".format((time.time()-t1)/60))
    main_sweep.update_values(new_points)
    exp.nTron_durations = new_points[:,0]
    exp.nTron_voltages = new_points[:,1]
    exp.setup_arb()

t2 = time.time()
exp.shutdown_instruments()
print("Total points: {}".format(len(points)))
print("Total elapsed time: {} min.".format((t2-t1)/60))

C:\Users\qlab\Anaconda3\lib\site-packages\sklearn\cluster\k_means_.py:821: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=10
  n_jobs=self.n_jobs)


21 new points added.
Elapsed time: 1.162315599123637 min.

45 new points added.
Elapsed time: 1.4258579333623251 min.

58 new points added.
Elapsed time: 1.9693557381629945 min.

92 new points added.
Elapsed time: 2.6639538288116453 min.

132 new points added.
Elapsed time: 3.7447510957717896 min.

215 new points added.
Elapsed time: 5.292644361654918 min.

261 new points added.
Elapsed time: 7.813336634635926 min.

338 new points added.
Elapsed time: 10.896857110659282 min.

628 new points added.
Elapsed time: 14.924935992558797 min.

761 new points added.
Elapsed time: 22.63941003481547 min.

1033 new points added.
Elapsed time: 32.18842465082805 min.

1805 new points added.
Elapsed time: 45.578509187698366 min.

2302 new points added.
Elapsed time: 68.48931938807169 min.
Total points: 5489
Total elapsed time: 68.58105798959733 min.


In [8]:
plt.show()

In [8]:
points, mean = sw.load_switching_data(wr.filename, start_state=0, display=True)
fig_sw = sw.phase_diagram_mesh(points, mean,
                              title = "nTron - P to AP",
                              xlabel = "Fall Time (s)",
                              ylabel = "Peak Voltage (V)")

C:\Users\qlab\Anaconda3\lib\site-packages\sklearn\cluster\k_means_.py:821: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=10
  n_jobs=self.n_jobs)


In [9]:
# Reset failures
points, fails = sw.load_switching_data(wr.filename, failure=True, start_state=0)
fig_fail = sw.phase_diagram_mesh(points, fails,
                              title = "nTron - P to AP - Failures",
                              xlabel = "Fall Time (s)",
                              ylabel = "Peak Voltage (V)")
plt.show()

C:\Users\qlab\Anaconda3\lib\site-packages\sklearn\cluster\k_means_.py:821: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=10
  n_jobs=self.n_jobs)


In [11]:
# For evaluation of adaptive method, plot the mesh
mesh, scale_factors = sw.scaled_Delaunay(points)
fig_mesh = sw.phase_diagram_mesh(points, mean, shading='gouraud')
plt.triplot(mesh.points[:,0]/scale_factors[0],
            mesh.points[:,1]/scale_factors[1], mesh.simplices.copy());
plt.show()

In [ ]:
points2, mean2 = sw.load_switching_data(wr.filename, start_state=1) # threshold = 130e+3*3e-6)
fig2 = sw.phase_diagram_mesh(points2, mean2)
plt.show()

C:\Users\qlab\Anaconda3\lib\site-packages\sklearn\cluster\k_means_.py:821: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=10
  n_jobs=self.n_jobs)


In [12]:
clusters = sw.cluster(mean, display=True)

C:\Users\qlab\Anaconda3\lib\site-packages\sklearn\cluster\k_means_.py:821: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=10
  n_jobs=self.n_jobs)


In [13]:
plt.show()

In [19]:
importlib.reload(sw)

<module 'analysis.switching' from 'C:\\Users\\qlab\\Documents\\GitHub\\pycontrol\\analysis\\switching.py'>

In [8]:
f = h5shell(wr.filename,'r')
f.close()

In [9]:
f

<h5shell>-<Closed HDF5 file>